---
execute:
  eval: true
  echo: true
  output: true
title: "object recognition"
---

- code examples [huggingface](https://huggingface.co/tasks){target="_blank"}
- download [jupyter notebook](pyws04-2-image-analysis.ipynb){target="_blank"}

In [ ]:
# run inside google colab
#!git clone https://github.com/cca-cce/osm-cca-cv.git
#!pip install -q timm

## image classification 

In [ ]:
from transformers import pipeline
clf = pipeline("image-classification")
clf("/content/osm-cca-cv/res/img/image_prefix-000.png")

#[{'label': 'tabby cat', 'score': 0.731},
#...
#]

## object recognition 

In [ ]:
from transformers import pipeline

model = pipeline("object-detection")
model("/content/osm-cca-cv/res/img/image_prefix-000.png")
# [{'label': 'blanket',
#  'mask': mask_string,
#  'score': 0.917},
#...]

## image, video to text 

In [ ]:
from transformers import pipeline

captioner = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")
#captioner("https://huggingface.co/datasets/Narsil/image_dummy/resolve/main/parrots.png")
captioner("https://raw.githubusercontent.com/cca-cce/osm-cca-cv/0a62207580cf63bc808ec825669af8d6f2eedf16/res/img/image_prefix-000.png")
## [{'generated_text': 'two birds are standing next to each other '}]

## pixtral

In [ ]:
# install dependencies
#pip install --upgrade vllm
#pip install --upgrade mistral_common

In [ ]:
from vllm import LLM
from vllm.sampling_params import SamplingParams

model_name = "mistralai/Pixtral-12B-2409"

sampling_params = SamplingParams(max_tokens=8192)

llm = LLM(model=model_name, tokenizer_mode="mistral")

prompt = "Describe this image in one sentence."
image_url = "https://raw.githubusercontent.com/cca-cce/osm-cca-cv/0a62207580cf63bc808ec825669af8d6f2eedf16/res/img/image_prefix-000.png"

messages = [
    {
        "role": "user",
        "content": [{"type": "text", "text": prompt}, {"type": "image_url", "image_url": {"url": image_url}}]
    },
]

outputs = llm.chat(messages, sampling_params=sampling_params)

print(outputs[0].outputs[0].text)

## testing.. on your own risk :)

To summarize and describe the visual content of a short video without audio using Python, you can follow these main steps:

1. **Extract frames from the video.**
2. **Select key frames that represent the content effectively.**
3. **Use an image captioning model to generate descriptions of the frames.**
4. **Combine the frame descriptions into a coherent summary.**

Below is a detailed explanation of each step along with the necessary code.

---

### 1. Extract Frames from the Video

*We use the `OpenCV` library to extract frames from the video file. Alternatively, `moviepy` can also be used for frame extraction.*

In [ ]:
import cv2
import os

# Define the path to the video file
video_file = 'path_to_your_video.mp4'

# Create a directory to store the extracted frames
frames_dir = 'extracted_frames'
os.makedirs(frames_dir, exist_ok=True)

# Load the video using OpenCV
cap = cv2.VideoCapture(video_file)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Decide on the interval between frames to extract (e.g., every 1 second)
frame_interval = int(fps)  # Adjust this value as needed

frame_number = 0
extracted_frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_number % frame_interval == 0:
        frame_path = os.path.join(frames_dir, f'frame_{frame_number}.jpg')
        cv2.imwrite(frame_path, frame)
        extracted_frames.append(frame_path)
    
    frame_number += 1

cap.release()
cv2.destroyAllWindows()

print(f"Extracted {len(extracted_frames)} frames.")

**Notes:**

- **Frame Extraction Interval:**
  - The `frame_interval` determines how frequently frames are extracted. For example, extracting one frame per second.
  - Adjust `frame_interval` based on the video's FPS and the level of detail you need.

- **Saving Frames:**
  - Frames are saved as JPEG images in the `extracted_frames` directory.

---

### 2. Use an Image Captioning Model to Generate Descriptions

*We use the `transformers` library from Hugging Face along with a pre-trained image captioning model to generate descriptions for each extracted frame.*

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

# Load the pre-trained image captioning model and processor
model_name = "nlpconnect/vit-gpt2-image-captioning"
model = VisionEncoderDecoderModel.from_pretrained(model_name)
feature_extractor = ViTImageProcessor.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate caption for an image
def generate_caption(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    # Generate caption
    with torch.no_grad():
        output_ids = model.generate(pixel_values, max_length=50, num_beams=5)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return caption

# Generate captions for all extracted frames
frame_captions = []

for frame_path in extracted_frames:
    caption = generate_caption(frame_path)
    frame_captions.append({'frame': frame_path, 'caption': caption})
    print(f"{frame_path}: {caption}")

**Notes:**

- **Model Selection:**
  - We use the `nlpconnect/vit-gpt2-image-captioning` model, which is a Vision Transformer (ViT) encoder and GPT-2 decoder.
  - You can explore other models on Hugging Face Hub if needed.

- **Device Configuration:**
  - The code checks for GPU availability and uses it if possible for faster processing.

- **Caption Generation:**
  - The `generate_caption` function processes an image and generates a caption.
  - Adjust parameters like `max_length` and `num_beams` for different results.

---

### 3. Combine Captions into a Coherent Summary

*We combine the generated captions to form a coherent summary of the video's visual content.*

In [ ]:
# Combine captions into a summary
summary_text = ' '.join([item['caption'] for item in frame_captions])

print("\nVideo Summary:")
print(summary_text)

**Notes:**

- **Simple Concatenation:**
  - We concatenate the captions to create a rough summary.
  - This method may result in repetitive or disjointed text.

- **Advanced Summarization (Optional):**
  - For a more coherent summary, you can use a text summarization model to refine the combined captions.

In [ ]:
from transformers import pipeline

# Initialize the summarization pipeline
summarizer = pipeline('summarization')

# Summarize the combined captions
final_summary = summarizer(summary_text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']

print("\nFinal Summary:")
print(final_summary)

**Notes:**

- **Summarization Model:**
  - The `summarization` pipeline uses a pre-trained model to condense the text.

- **Adjusting Parameters:**
  - Modify `max_length` and `min_length` to control the length of the final summary.

---

### Complete Code Example

Putting it all together, here's the complete code:

In [ ]:
import cv2
import os
import torch
from PIL import Image
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer, pipeline

# Step 1: Extract Frames from the Video
video_file = 'path_to_your_video.mp4'
frames_dir = 'extracted_frames'
os.makedirs(frames_dir, exist_ok=True)

cap = cv2.VideoCapture(video_file)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps)  # Extract one frame per second
frame_number = 0
extracted_frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_number % frame_interval == 0:
        frame_path = os.path.join(frames_dir, f'frame_{frame_number}.jpg')
        cv2.imwrite(frame_path, frame)
        extracted_frames.append(frame_path)
    
    frame_number += 1

cap.release()
cv2.destroyAllWindows()

print(f"Extracted {len(extracted_frames)} frames.")

# Step 2: Generate Captions for Each Frame
model_name = "nlpconnect/vit-gpt2-image-captioning"
model = VisionEncoderDecoderModel.from_pretrained(model_name)
feature_extractor = ViTImageProcessor.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_caption(image_path):
    image = Image.open(image_path).convert("RGB")
    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    with torch.no_grad():
        output_ids = model.generate(pixel_values, max_length=50, num_beams=5)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return caption

frame_captions = []

for frame_path in extracted_frames:
    caption = generate_caption(frame_path)
    frame_captions.append({'frame': frame_path, 'caption': caption})
    print(f"{frame_path}: {caption}")

# Step 3: Combine Captions into a Summary
summary_text = ' '.join([item['caption'] for item in frame_captions])

# Optional: Summarize the Combined Captions
summarizer = pipeline('summarization')
final_summary = summarizer(summary_text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']

print("\nFinal Summary:")
print(final_summary)

---

### Additional Considerations

- **Installing Dependencies:**

  Ensure that you have all the required libraries installed:

  ```bash
  pip install opencv-python
  pip install transformers
  pip install torch  # For GPU support, install torch appropriate for your CUDA version
  pip install pillow
  ```

- **Frame Selection:**

  - **Dynamic Frame Selection:**
    - Instead of extracting frames at fixed intervals, you can implement shot detection or scene change detection to extract frames when the content changes significantly.
    - Libraries like `scenedetect` can help with this.

- **Processing Time:**

  - Image captioning models can be computationally intensive.
  - Processing time will increase with the number of frames and the complexity of the model.
  - Using a GPU significantly speeds up the process.

- **Model Limitations:**

  - The quality of the generated captions depends on the model's capabilities.
  - Pre-trained models may not accurately describe all types of content.
  - For domain-specific videos, consider fine-tuning a model on relevant datasets.

- **Enhancing Summarization:**

  - **Text Preprocessing:**
    - Remove duplicate captions or captions with minimal information before summarization.
  - **Semantic Understanding:**
    - Use natural language understanding techniques to better combine captions.

- **Privacy and Compliance:**

  - Be cautious when processing videos containing personal or sensitive information.
  - Ensure compliance with data protection regulations.

---

### Conclusion

By following these steps, you can extract frames from a video without audio, generate descriptions of the visual content using an image captioning model, and combine those descriptions into a textual summary. This approach leverages powerful pre-trained models available through the Hugging Face Transformers library and allows you to create summaries of visual content programmatically.

**References:**

- **OpenCV Documentation:** [https://docs.opencv.org/](https://docs.opencv.org/)
- **Hugging Face Transformers Documentation:** [https://huggingface.co/transformers/](https://huggingface.co/transformers/)
- **Image Captioning Models:** [https://huggingface.co/models?pipeline_tag=image-to-text](https://huggingface.co/models?pipeline_tag=image-to-text)
- **Scene Detection with PySceneDetect:** [https://pyscenedetect.readthedocs.io/](https://pyscenedetect.readthedocs.io/)

---

**Example Output:**

Assuming we have a video and we run the code, the output might look like:

```
Extracted 10 frames.
extracted_frames/frame_0.jpg: a group of people standing in a room.
extracted_frames/frame_30.jpg: a man holding a microphone.
extracted_frames/frame_60.jpg: a woman sitting at a desk with a laptop.
...
Final Summary:
A group of people are standing in a room. A man is holding a microphone. A woman is sitting at a desk with a laptop. ...
```

---

**Note:**

- Replace `'path_to_your_video.mp4'` with the actual path to your video file.
- Adjust the `frame_interval` based on the video's length and desired granularity.
- Ensure you have sufficient system resources to run the models, especially if processing longer videos or using higher-resolution frames.

---

By automating the process of generating textual descriptions from video frames, you can create summaries of videos without audio, which is particularly useful for silent surveillance footage, visual-only content, or scenarios where audio data is unavailable or unusable.